# Prepare Data From CSV Recording

## Load File

Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.

The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.

The new trimmed file is written to a directory (/trim_output), for further manipulation and loading into the model.


In [21]:
import numpy as np 
import pandas as pd

recording_path = "../recordings/"
output_path = "../trim_output/"
file_name = input("Input File Name")

#Read in CSV

try:
    dataframe = pd.read_csv(recording_path + file_name + ".csv")
except:
    print("Error Reading File: Check Spelling and Try Again")

# Seperate each tracker to seperate dataframe
HMD = dataframe.iloc[:, 1:4]
controller_1 = dataframe.iloc[:, 7:10]
controller_2 = dataframe.iloc[:, 37:40]
left_foot = dataframe.iloc[:, 67:70]
right_foot = dataframe.iloc[:, 73:76]
waist = dataframe.iloc[:, 79:82]

# Join all trackers together
joined = pd.concat([HMD, controller_1, controller_2,
                   waist, left_foot, right_foot], axis=1)

# set new column headers
joined.columns = [
    "head_x",
    "head_y",
    "head_z",
    "r_controller_x",
    "r_controller_y", 
    "r_controller_z",
    "l_controller_x",
    "l_controller_y",
    "l_controller_z",
    "waist_x",
    "waist_y",
    "waist_z",
    "r_foot_x",
    "r_foot_y",
    "r_foot_z",
    "l_foot_x",
    "l_foot_y",
    "l_foot_z"
]


# output to new csv
output_file = output_path + file_name + "_trimmed.csv"
joined.to_csv(output_file, index=False)

print(file_name + " output to " + output_path)


Input File Name walking_2_test


Error Reading File: Check Spelling and Try Again


ValueError: Length mismatch: Expected axis has 6 elements, new values have 18 elements

# Data Normalization

## Data Scaling

The new CSV is loaded into memory, chosen through an input prompt
The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).
These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"


#read in formatted CSV
def ReadCSV():
    file_name = input("Input File Name")
    try:
        dataframe = pd.read_csv(output_path + file_name + ".csv")
        print("Dataframe created")
    except:
        print("Error Reading File")
    return dataframe

def SplitFeaturesLabels(dataframe):
    x = dataframe.iloc[:, 0:9]
    y = dataframe.iloc[:, 9:18]
    return x, y

#Load data into Numpy array
def LoadArray(x, y):
    x_array = np.array(x)
    y_array = np.array(y)
    return x_array, y_array


def NormalizeValues (x, y):
    x =  np.divide(x, 100)
    y =  np.divide(y, 100)
    return x, y

def SampleSize(x, y):
    x_samples = x[0:900,:]
    y_samples = y[0:900,:]
    return x_samples, y_samples

    
    

    





# Create Training Data

In [27]:
    
#load train data from csv
train_dataframe = ReadCSV()

#split features and labels into seperate dataframes
x_train_df, y_train_df = SplitFeaturesLabels(train_dataframe)

#convert features and labels to numpy array
x_train, y_train = LoadArray(x_train_df, y_train_df)

#Divide values in array by 100
x_train_normalized, y_train_normalized = NormalizeValues(x_train, y_train)


x_samples, y_samples = SampleSize(x_train_normalized, y_train_normalized)

print(x_samples.shape, x_samples)
print(y_samples.shape, y_samples)

#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)





Input File Name walking_1_train


Dataframe created
(900, 9) [[ 0.00396347  1.55321396 -0.09726781 ... -0.17044104  0.79216263
  -0.12728346]
 [ 0.00396347  1.55321396 -0.0974095  ... -0.1698889   0.79222771
  -0.12750142]
 [ 0.00419568  1.55321396 -0.09760312 ... -0.16916578  0.79246605
  -0.12798414]
 ...
 [-0.02651245  1.59055893 -0.13678305 ... -0.11887306  0.75262321
   0.13336996]
 [-0.03070768  1.59075287 -0.13581562 ... -0.13185926  0.75569618
   0.13421907]
 [-0.03509516  1.5909549  -0.13504914 ... -0.14492013  0.75884453
   0.13558317]]
(900, 9) [[ 0.03550047  0.99325623 -0.06203985 ... -0.17026985  0.09899879
   0.01698667]
 [ 0.03546809  0.99353798 -0.06230913 ... -0.17021811  0.09892341
   0.01698667]
 [ 0.0358079   0.99352867 -0.06240509 ... -0.17014109  0.09882551
   0.01698667]
 ...
 [-0.07211315  1.00282028 -0.1262004  ... -0.08610182  0.09165998
   0.05806892]
 [-0.07784638  1.00439491 -0.1233117  ... -0.08573814  0.09138713
   0.05814877]
 [-0.08183085  1.0055632  -0.12043599 ... -0.08560102  0.09083

# Reshape Training Data

In [59]:

def ReshapeData(x, y):
    x_reshaped = np.expand_dims(x, axis=1)
    y_reshaped = np.expand_dims(y, axis=1)

    return x_reshaped, y_reshaped


In [60]:
x_train, y_train = ReshapeData(x_samples, y_samples)

print(x_train.shape, y_train.shape)

print(x_train.shape[1])

print(x_train.shape[2])


(900, 1, 9) (900, 1, 9)
1
9


# Create Test / Validation Data

In [64]:
test_dataframe = ReadCSV()

#split features and labels into seperate dataframes
x_test_df, y_test_df = SplitFeaturesLabels(test_dataframe)

#convert features and labels to numpy array
x_test, y_test = LoadArray(x_test_df, y_test_df)

#Divide values in array by 100
x_test_normalized, y_test_normalized = NormalizeValues(x_test, y_test)

x_test_samples, y_test_samples = SampleSize(x_test_normalized, y_test_normalized)

x_test, y_test = ReshapeData(x_test_samples, y_test_samples)

print(x_test.shape, y_test.shape)

print(x_test.shape[1])

Input File Name walking_2_test


Dataframe created
(900, 1, 9) (900, 1, 9)
1


# Model Creation and Training

In [61]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
import tensorflow as tf
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout

tf.keras.backend.set_floatx('float64')


model = Sequential()
model.add(GRU(9, return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(9, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(9, activation = "relu"))

model.compile(loss='mse', optimizer='adam')

print ('model compiled')

print (model.summary())

ValueError: Input 0 of layer "gru_15" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 9)

In [72]:

model.fit(x_train, y_train, epochs=20,batch_size=300)

Epoch 1/20
3/3 [==============================] - 0s 14ms/step - loss: 0.0818
Epoch 2/20
3/3 [==============================] - 0s 10ms/step - loss: 0.0815
Epoch 3/20
3/3 [==============================] - 0s 8ms/step - loss: 0.0816
Epoch 4/20
3/3 [==============================] - 0s 9ms/step - loss: 0.0816
Epoch 5/20
3/3 [==============================] - 0s 12ms/step - loss: 0.0815
Epoch 6/20
3/3 [==============================] - 0s 11ms/step - loss: 0.0814
Epoch 7/20
3/3 [==============================] - 0s 10ms/step - loss: 0.0815
Epoch 8/20
3/3 [==============================] - 0s 10ms/step - loss: 0.0815
Epoch 9/20
3/3 [==============================] - 0s 10ms/step - loss: 0.0818
Epoch 10/20
3/3 [==============================] - 0s 11ms/step - loss: 0.0815
Epoch 11/20
3/3 [==============================] - 0s 10ms/step - loss: 0.0813
Epoch 12/20
3/3 [==============================] - 0s 9ms/step - loss: 0.0813
Epoch 13/20
3/3 [==============================] - 0s 11ms/step 

In [71]:
model.evaluate(x_test, y_test, batch_size=32)

29/29 [==============================] - 0s 1ms/step - loss: 0.1009


0.10086438182327483

In [73]:
predictions = model.predict(x_test)
print("predictions shape:", predictions.shape)
print(predictions)
pd.DataFrame(predictions)


predictions shape: (900, 9)
[[0.         0.9810058  0.14005643 ... 0.         0.11163873 0.15246763]
 [0.         0.98096037 0.14006484 ... 0.         0.11164343 0.15248524]
 [0.         0.98087372 0.14007096 ... 0.         0.11164781 0.1525026 ]
 ...
 [0.         0.96768579 0.13832566 ... 0.         0.1097853  0.15221212]
 [0.         0.96873128 0.1384157  ... 0.         0.10981201 0.15215462]
 [0.         0.96962389 0.13848058 ... 0.         0.10982646 0.15209785]]


,0,1,2,3,4,5,6,7,8
0,0.0,0.981006,0.140056,0.077319,0.117531,0.146038,0.0,0.111639,0.152468
1,0.0,0.980960,0.140065,0.077290,0.117536,0.146038,0.0,0.111643,0.152485
2,0.0,0.980874,0.140071,0.077259,0.117538,0.146039,0.0,0.111648,0.152503
3,0.0,0.980776,0.140080,0.077220,0.117540,0.146038,0.0,0.111654,0.152524
4,0.0,0.980627,0.140085,0.077181,0.117539,0.146039,0.0,0.111661,0.152546
...,...,...,...,...,...,...,...,...,...
895,0.0,0.966955,0.138310,0.082910,0.114795,0.151546,0.0,0.109780,0.152388
896,0.0,0.967293,0.138316,0.082948,0.114789,0.151433,0.0,0.109783,0.152299
897,0.0,0.967686,0.138326,0.082994,0.114788,0.151322,0.0,0.109785,0.152212
898,0.0,0.968731,0.138416,0.083001,0.114817,0.151169,0.0,0.109812,0.152155
